# Water visualizations data
### Python libraries

In [7]:
import geopandas as gpd
import shapely.wkb 
import requests

/Users/ikersanchez/anaconda3/envs/geoenv/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(


### Utils
**df_from_carto**

In [8]:
def df_from_carto(account, query):
    """
    It gets data by querying a carto table and converts it into a GeoDataFrame.
    """
    urlCarto = f"https://{account}.carto.com/api/v2/sql"
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    df = gpd.GeoDataFrame(data.get("rows"))
    if 'the_geom' in df.columns:
        # Change geometry from WKB to WKT format
        df['geometry'] = df.apply(lambda x: shapely.wkb.loads(x['the_geom'],hex=True), axis=1 )
        df.drop(columns='the_geom', inplace=True)
        if 'the_geom_webmercator' in df.columns:
            df.drop(columns=['the_geom_webmercator'], inplace=True)
        df.crs = {'init': 'epsg:4326'}
        df = df.to_crs({'init': 'epsg:4326'})
        
    return df, r.url

## Country ranking widget
- Check this [doc](https://onewri-my.sharepoint.com/:w:/g/personal/rutger_hofste_wri_org/EWbmrDU6VHdEg_o85iiaI5QB-2pdtpaVLEv_xwEQ2cs9nQ?rtime=Tt0B_9Po1kg) for a description.

- Check this [site](https://www.wri.org/applications/aqueduct/country-rankings/?indicator=bws) for an example.

**Weighting used:**

|Indicator | total gross withdrawal  | domestic gross withdrawal  | industrial gross withdrawal  | irrigation gross withdrawal | livestock gross withdrawal | Population | 
|:---|:---|:---|:---|:---|:---|:---|
| Baseline Water Stress | 1 | 1 | 1 | 1 | 1 | - | 
| Drought Risk          | 1 | 1 | 1 | 1 | 1 | - | 
| Riverine Flood Risk   | - | - | - | - | - | 1 | 

**Parametrization:**

|Indicator | indicator_name  | 
|:---|:---|
| Baseline Water Stress | bws |
| Drought Risk          | drr |
| Riverine Flood Risk   | rfr |


|Weight | weight_name  | 
|:---|:---|
| Total gross withdrawal      | Tot |
| Domestic gross withdrawal   | Dom |
| Industrial gross withdrawal | Ind |
| Irrigation gross withdrawal | Irr |
| Livestock gross withdrawal  | Liv |
| Population                  | Pop |

`Sql Template:`
```sql
SELECT gid_0 as iso, name_0 as country, indicator_name, weight, score, score_ranked
FROM aqueduct_results_v01_country_v06
WHERE indicator_name = {{indicator}} and weight in ({{weights}}) and score != -9999
ORDER BY score_ranked
```

`params_config:`
```json
"params_config": [
    {
        "required": true,
        "key": "indicator"
    },
    {
        "required": true,
        "key": "weights"
    }
],
```

`Sql Example:`
- {{indicator}} = 'bws'
- {{weights}} = 'Tot'

In [9]:
account = 'wri-rw'

query = "SELECT gid_0 as iso, name_0 as country, indicator_name, weight, score, score_ranked FROM aqueduct_results_v01_country_v06 WHERE indicator_name = 'bws' and weight in ('Tot') and score != -9999 ORDER BY score_ranked"

df, url = df_from_carto(account, query)
url

'https://wri-rw.carto.com/api/v2/sql?q=SELECT+gid_0+as+iso%2C+name_0+as+country%2C+indicator_name%2C+weight%2C+score%2C+score_ranked+FROM+aqueduct_results_v01_country_v06+WHERE+indicator_name+%3D+%27bws%27+and+weight+in+%28%27Tot%27%29+and+score+%21%3D+-9999+ORDER+BY+score_ranked'

In [10]:
df.head()

,iso,country,indicator_name,weight,score,score_ranked
0,QAT,Qatar,bws,Tot,4.973635,1
1,ISR,Israel,bws,Tot,4.820170,2
2,LBN,Lebanon,bws,Tot,4.816692,3
3,IRN,Iran,bws,Tot,4.567056,4
4,JOR,Jordan,bws,Tot,4.561961,5


## Province ranking widget

`Sql Template:`
```sql
SELECT gid_0 as iso, name_0 as country, name_1 as province, indicator_name, weight, score, score_ranked 
FROM aqueduct_results_v01_province_v06
WHERE indicator_name = {{indicator}} and gid_0 = {{iso}} and weight in ({{weights}}) and score != -9999
ORDER BY score_ranked 
```

`params_config:`
```json
"params_config": [
    {
        "required": true,
        "key": "indicator"
    },
    {
        "required": true,
        "key": "iso"
    },
    {
        "required": true,
        "key": "weights"
    }
],
```

`Sql Example:`
- {{indicator}} = 'bws'
- {{iso}} = 'ESP'
- {{weight}} = 'Tot'

In [11]:
account = 'wri-rw'

query = query = "SELECT gid_0 as iso, name_0 as country, name_1 as province, indicator_name, weight, score, score_ranked FROM aqueduct_results_v01_province_v06 WHERE indicator_name = 'bws' and gid_0 = 'ESP' and weight in ('Tot') and score != -9999 ORDER BY score_ranked"

df, url = df_from_carto(account, query)
url

'https://wri-rw.carto.com/api/v2/sql?q=SELECT+gid_0+as+iso%2C+name_0+as+country%2C+name_1+as+province%2C+indicator_name%2C+weight%2C+score%2C+score_ranked+FROM+aqueduct_results_v01_province_v06+WHERE+indicator_name+%3D+%27bws%27+and+gid_0+%3D+%27ESP%27+and+weight+in+%28%27Tot%27%29+and+score+%21%3D+-9999+ORDER+BY+score_ranked'

In [12]:
df

,iso,country,province,indicator_name,weight,score,score_ranked
0,ESP,Spain,Región de Murcia,bws,Tot,4.902860,105
1,ESP,Spain,Andalucía,bws,Tot,4.659056,189
2,ESP,Spain,Comunidad de Madrid,bws,Tot,4.582692,213
3,ESP,Spain,Islas Baleares,bws,Tot,4.401156,265
4,ESP,Spain,Castilla-La Mancha,bws,Tot,4.317735,281
5,ESP,Spain,Comunidad Valenciana,bws,Tot,3.612639,519
6,ESP,Spain,Castilla y León,bws,Tot,3.566634,532
7,ESP,Spain,La Rioja,bws,Tot,3.311637,622
8,ESP,Spain,Islas Canarias,bws,Tot,3.195637,661
9,ESP,Spain,Extremadura,bws,Tot,3.191919,663
